<a href="https://colab.research.google.com/github/FantySantos/Estatistica_e_LLMs/blob/main/Atividade_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 1 — Z-score e Binomial Distribution Sample distribution and sampling distribution TLC p-value

Atividade apresentada como parte da avaliação da disciplina Tópicos especiais: Introdução à Estatística e LLMs ministrada pelo professor Claudomiro de Souza de Sales Junior para o curso
de Sistemas de Informação do Instituto de Ciências Exatas
e Naturais da Universidade Federal do Pará - Campus
Guamá.

Aluna: Fantiny Santos dos Santos 202211140005

---

## Preparação do Ambiente

In [35]:
import pandas as pd
import kagglehub
import matplotlib.pyplot as plt
import plotly.express as px
pd.options.plotting.backend = "plotly"

## Parte 1 — Z-score e Binomial Distribution

### Questão A - Busque um dataset e faça histogramas de duas features para cada classe no dataset, conforme procedimento mostrado em sala. Faça dois conjuntos de histogramas com tamanhos de bins diferentes. Discuta os histogramas do ponto de vista de classificação usando aprendizado de máquina e também discuta os tamanhos dos bins usados. Não esqueça do contexto sobre o dataset usado, com a explicação sobre ele, suas classes e features.

#### Visão Geral do Dataset: Pima Indians Diabetes

##### Leitura do Dataset

[Smith, J.W., Everhart, J.E., Dickson, W.C., Knowler, W.C., & Johannes, R.S. (1988). Using the ADAP learning algorithm to forecast the onset of diabetes mellitus. In Proceedings of the Symposium on Computer Applications and Medical Care (pp. 261--265). IEEE Computer Society Press.](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database)

In [2]:
df = kagglehub.dataset_load(
    kagglehub.KaggleDatasetAdapter.PANDAS,
    "uciml/pima-indians-diabetes-database",
    "diabetes.csv",
)

df.head()

Using Colab cache for faster access to the 'pima-indians-diabetes-database' dataset.


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


##### Descrição:

Este dataset contém informações clínicas de mulheres da etnia Pima, com idade mínima de 21 anos, coletadas pelo Instituto Nacional de Diabetes e Doenças Digestivas e Renais. O objetivo principal é prever a presença de diabetes com base em várias medições fisiológicas e metabólicas. As variáveis incluem indicadores como nível de glicose, pressão arterial, IMC, idade, entre outros fatores relacionados ao risco de diabetes tipo 2.

##### O que é a diabetes?

A diabetes é uma condição em que o nível de açúcar no sangue fica acima do normal porque o corpo não produz insulina suficiente ou não consegue utilizá-la corretamente. A insulina é o hormônio que permite que a glicose entre nas células e seja usada como energia; quando esse processo falha, o açúcar se acumula no sangue e pode causar diversos problemas de saúde ao longo do tempo.

Existem dois tipos principais de diabetes. No tipo 1, o sistema imunológico destrói as células do pâncreas que produzem insulina, tornando necessária a aplicação diária do hormônio. Já no tipo 2, o corpo tem dificuldade em usar a insulina de forma adequada ou a produz em menor quantidade, sendo o tipo mais comum e geralmente associado ao estilo de vida e fatores genéticos.

##### Dicionário das Features

| Feature                         | Descrição                                                               |
|----------------------------------|-------------------------------------------------------------------------|
| Pregnancies                      | Número de vezes grávida                                                 |
| Glucose                          | Concentração de glicose plasmática 2h após teste de tolerância oral     |
| BloodPressure                    | Pressão arterial diastólica (mm Hg)                                     |
| SkinThickness                    | Espessura da dobra cutânea do tríceps (mm)                             |
| Insulin                          | Nível sérico de insulina após 2 horas (mu U/ml)                        |
| BMI                              | Índice de Massa Corporal = peso (kg) / altura² (m²)                    |
| DiabetesPedigreeFunction         | Função de Pedigree de Diabetes (Quantificar o risco genético)                    |
| Age                              | Idade da paciente (anos)                                                |
| Outcome                          | Variável alvo: 268 dos 768 casos são 1; os demais são 0                 |


##### Dimensão e Tipo de Dados

O df.info() ajuda a identificar os tipos dos dados, o que é importante caso alguma feature precise ser convertida, por exemplo, uma variável numérica que esteja como string. No nosso dataset, a maioria das colunas é inteira, enquanto apenas BMI e DiabetesPedigreeFunction são floats, e no nosso caso todos os tipos já estão corretos. O df.info() também mostra o tamanho do dataset, que aqui possui 768 registros e 9 colunas, o que é útil porque muitos registros podem deixar o treinamento mais lento e poucos podem prejudicar o aprendizado. Da mesma forma, um número alto de features pode afetar o desempenho de alguns algoritmos.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [39]:
from plotly.subplots import make_subplots

In [90]:
for bins in [70, 30]:
  fig_glicose = df.hist(color="Outcome", x="Glucose", text_auto=True, nbins=bins, labels={"Outcome": "Resultado", "Glucose": "Glicose"})
  fig_glicose.update_layout(title_text=f"Glicose - Bins={bins}", title_x=0.5)
  fig_glicose.update_yaxes(title_text="Frequência")
  fig_glicose.show()

In [91]:
for bins in [60, 30]:
  fig_imc = df.hist(color="Outcome", x="BMI", text_auto=True, nbins=bins, labels={"Outcome": "Resultado", "BMI": "IMC"})
  fig_imc.update_layout(title_text=f"IMC - Bins={bins}", title_x=0.5)
  fig_imc.update_yaxes(title_text="Frequência")
  fig_imc.show()